You will try to reproduce the bug of HDFS-15118 in this file. Referenced from tutorial: https://www.chameleoncloud.org/experiment/share/370ce99a-3e03-43e9-83e3-b61fd9692dc0 

In [1]:
import chi

chi.use_site("CHI@TACC")

# Change to your project (CHI-XXXXXX)
chi.set("project_name", "CHI-220969")

print(f'Using Project {chi.get("project_name")}')

KeyboardInterrupt: 

In [ ]:
import os
import keystoneauth1, blazarclient
from chi import lease

reservations = []
lease_node_type = "compute_skylake"

try:
    print("Creating lease...")
    lease.add_fip_reservation(reservations, count=1)
    lease.add_node_reservation(reservations, node_type=lease_node_type, count=1)

    start_date, end_date = lease.lease_duration(hours=3)

    l = lease.create_lease(
        f"{os.getenv('USER')}-power-management", 
        reservations, 
        start_date=start_date, 
        end_date=end_date
    )
    lease_id = l["id"]

    print("Waiting for lease to start ...")
    lease.wait_for_active(lease_id)
    print("Lease is now active!")
except keystoneauth1.exceptions.http.Unauthorized as e:
    print("Unauthorized.\nDid set your project name and and run the code in the first cell?")
except blazarclient.exception.BlazarClientException as e:
    print(f"There is an issue making the reservation. Check the calendar to make sure a {lease_node_type} node is available.")
    print("https://chi.uc.chameleoncloud.org/project/leases/calendar/host/")
    print(e)
except Exception as e:
    print("An unexpected error happened.")
    print(e)

In [ ]:
from chi import server

image = "CC-Ubuntu18.04-20201016"

s = server.create_server(
    f"{os.getenv('USER')}-power-management", 
    image_name=image,
    reservation_id=lease.get_node_reservation(lease_id)
)

print("Waiting for server to start ...")
server.wait_for_active(s.id)
print("Done")

In [ ]:
floating_ip = lease.get_reserved_floating_ips(lease_id)[0]
server.associate_floating_ip(s.id, floating_ip_address=floating_ip)

print(f"Waiting for SSH connectivity on {floating_ip} ...")
timeout = 60*2
import socket
import time
# Repeatedly try to connect via SSH.
start_time = time.perf_counter()
while True:
    try:
        with socket.create_connection((floating_ip, 22), timeout=timeout):
            print("Connection successful")
            break
    except OSError as ex:
        time.sleep(10)
        if time.perf_counter() - start_time >= timeout:
            print(f"After {timeout} seconds, could not connect via SSH. Please try again.")


In [ ]:
from chi import ssh

with ssh.Remote(floating_ip) as conn:
    # Upload the script
    conn.put("setup.sh")
    # Run the script
    conn.run("bash setup.sh")

In [ ]:
with ssh.Remote(floating_ip) as conn:
    # Upload the script
    conn.put("get_repo.sh")
    # Run the script
    conn.run("bash get_repo.sh")

In [ ]:
with ssh.Remote(floating_ip) as conn:
    # Upload the script
    conn.put("run_experiment.sh")
    # Run the script for 10 seconds, sample every 0.5 seconds 
    conn.run("bash run_experiment.sh")

In [ ]:
with ssh.Remote(floating_ip) as conn:
    # Upload the script
    conn.put("check_result.sh")
    # Run the script
    conn.run("bash check_result.sh")

In [ ]:
import matplotlib.pyplot as plt

# Data
number_of_namenodes = ["1", "2", "4", "8", "16"]
time_took = [3, 12, 23, 53, 118]

# Plot
plt.bar(number_of_namenodes, time_took, label="Before fixing", color='skyblue', width=0.8)

# Add labels and title
plt.xlabel('Number of NameNodes')
plt.ylabel('Time Took')
plt.title('Time Taken vs Number of NameNodes')

# Add legend
plt.legend()

# Show plot
plt.show()
# After the fix, the bars all have the same heights because we won't even go into the for loop in this case. 